In [9]:
!pip install jsonapi_client

Defaulting to user installation because normal site-packages is not writeable


In [10]:
# declarations:
biom = GALAXY_INPUTS['biom']
max_sample_count = GALAXY_INPUTS['count']

# rework max_sample_count.




In [11]:
from jsonapi_client import Session
import pandas as pd
import numpy as np
import requests
import os

###################
# Definie Functions
###################


sample_accessions = []



# retuns list of id`s for all Studies to Biom
def GetBiomSamplesByIds(biom_id: str):
    with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
        i = 0
        biomes_dfs = []
        for r in mgnify.iterate(f'biomes/{biom_id}/samples'):
            biom_df = pd.json_normalize(r.json)
            biom_df['url'] = str(r.links.self)
            biomes_dfs.append(biom_df)
            i += 1
            if(i == max_sample_count):
                break
        
    main_biomes_df = pd.concat(biomes_dfs)

    # main_biomes_df = main_biomes_df.dropna()

    return main_biomes_df

def GetAnalysisFromStudie(stydy_id):
    with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
        i = 0
        biomes_dfs = []
        for r in mgnify.iterate(f'studies/{stydy_id}/analyses'):
            biom_df = pd.json_normalize(r.json)
            biom_df['url'] = str(r.links.self)
            biomes_dfs.append(biom_df)
    if(biomes_dfs == []):
        return [None,None]
    main_biomes_df = pd.concat(biomes_dfs)

    # main_biomes_df = main_biomes_df.dropna()

    return [1,main_biomes_df]

df_samples = GetBiomSamplesByIds(biom)

df_samples

,type,id,attributes.sample-metadata,attributes.biosample,attributes.latitude,attributes.longitude,attributes.accession,attributes.collection-date,attributes.sample-desc,attributes.environment-biome,...,attributes.environment-material,attributes.sample-name,attributes.sample-alias,attributes.last-update,relationships.studies.data,relationships.biome.data.id,relationships.biome.data.type,url,attributes.analysis-completed,attributes.geo-loc-name
0,samples,ERS693376,"[{'key': 'geographic location (latitude)', 'va...",SAMEA3317944,48.7833,9.1833,ERS693376,2013-01-01,Wine Making Soil Sample,soil,...,soil,Wine_Dechlor1,Wine_Dechlor1,2023-09-15T16:34:43,"[{'id': 'MGYS00005519', 'type': 'studies'}, {'...",root:Environmental:Terrestrial:Soil:Agricultural,biomes,https://www.ebi.ac.uk/metagenomics/api/v1/samp...,NaN,NaN
0,samples,ERS744877,"[{'key': 'investigation type', 'value': 'metag...",SAMEA3446556,34.8500,148.5500,ERS744877,NaN,Glenrock site,Pasture,...,Soil,642,642,2023-08-06T16:36:59,"[{'id': 'MGYS00000498', 'type': 'studies'}]",root:Environmental:Terrestrial:Soil,biomes,https://www.ebi.ac.uk/metagenomics/api/v1/samp...,2015-07-22,Australia
0,samples,ERS744878,"[{'key': 'investigation type', 'value': 'metag...",SAMEA3446557,34.8500,148.5500,ERS744878,NaN,Glenrock site,Pasture,...,Soil,644,644,2023-08-06T16:36:13,"[{'id': 'MGYS00000498', 'type': 'studies'}]",root:Environmental:Terrestrial:Soil,biomes,https://www.ebi.ac.uk/metagenomics/api/v1/samp...,2015-07-22,Australia
0,samples,ERS744879,"[{'key': 'investigation type', 'value': 'metag...",SAMEA3446558,34.8500,148.5500,ERS744879,NaN,Glenrock site,Pasture,...,Soil,646,646,2023-08-06T16:35:29,"[{'id': 'MGYS00000498', 'type': 'studies'}]",root:Environmental:Terrestrial:Soil,biomes,https://www.ebi.ac.uk/metagenomics/api/v1/samp...,2015-07-22,Australia
0,samples,ERS744880,"[{'key': 'investigation type', 'value': 'metag...",SAMEA3446559,34.8500,148.5500,ERS744880,NaN,Glenrock site,Pasture,...,Soil,651,651,2023-08-06T16:34:45,"[{'id': 'MGYS00000498', 'type': 'studies'}]",root:Environmental:Terrestrial:Soil,biomes,https://www.ebi.ac.uk/metagenomics/api/v1/samp...,2015-07-22,Australia
0,samples,ERS744881,"[{'key': 'investigation type', 'value': 'metag...",SAMEA3446560,34.8500,148.5500,ERS744881,NaN,Glenrock site,Pasture,...,Soil,655,655,2023-08-06T16:34:00,"[{'id': 'MGYS00000498', 'type': 'studies'}]",root:Environmental:Terrestrial:Soil,biomes,https://www.ebi.ac.uk/metagenomics/api/v1/samp...,2015-07-22,Australia
0,samples,ERS744882,"[{'key': 'investigation type', 'value': 'metag...",SAMEA3446561,34.8500,148.5500,ERS744882,NaN,Glenrock site,Pasture,...,Soil,657,657,2023-08-06T16:33:14,"[{'id': 'MGYS00000498', 'type': 'studies'}]",root:Environmental:Terrestrial:Soil,biomes,https://www.ebi.ac.uk/metagenomics/api/v1/samp...,2015-07-22,Australia
0,samples,ERS744883,"[{'key': 'investigation type', 'value': 'metag...",SAMEA3446562,34.8500,148.5500,ERS744883,NaN,Glenrock site,Pasture,...,Soil,661,661,2023-08-06T16:32:29,"[{'id': 'MGYS00000498', 'type': 'studies'}]",root:Environmental:Terrestrial:Soil,biomes,https://www.ebi.ac.uk/metagenomics/api/v1/samp...,2015-07-22,Australia
0,samples,ERS744884,"[{'key': 'investigation type', 'value': 'metag...",SAMEA3446563,34.8500,148.5500,ERS744884,NaN,Glenrock site,Pasture,...,Soil,663,663,2023-08-06T16:31:41,"[{'id': 'MGYS00000498', 'type': 'studies'}]",root:Environmental:Terrestrial:Soil,biomes,https://www.ebi.ac.uk/metagenomics/api/v1/samp...,2015-07-22,Australia
0,samples,ERS744885,"[{'key': 'investigation type', 'value': 'metag...",SAMEA3446564,34.8500,148.5500,ERS744885,NaN,Glenrock site,Woodlands,...,Soil,668,668,2023-08-06T16:30:58,"[{'id': 'MGYS00000498', 'type': 'studies'}]",root:Environmental:Terrestrial:Soil,biomes,https://www.ebi.ac.uk/metagenomics/api/v1/samp...,2015-07-22,Australia


In [12]:
sample_accessions = (df_samples['attributes.accession'].to_list())

studies = df_samples['relationships.studies.data'].to_list()

study_analaysises = []

studies_without_analysis = []

studies_Ids = []

for study in studies:
    print(study)
    # list of studies for sample
    s_ids_sample = []
    for s in str(study).split("'"):
        if('MGYS' in s):
            s_ids_sample.append(s) 
    # for each study
    for st in s_ids_sample:
        print(f'statt {st}')
        # test if we havent checked study
        if(st in studies_Ids):
            print(f'skipped {st} -duplicate- E')
            # we allready have an analysis
            break
        if(st not in studies_without_analysis):
            print(f'get analysis from {st}')
            study_analaysis = GetAnalysisFromStudie(st)
            if(study_analaysis[0] == None):
                # discard if studie has no analysis
                studies_without_analysis.append(st)
                print(f'{st} has no analysis')
                continue
            # keep if study has analysis
            studies_Ids.append(st)
            study_analaysises.append(study_analaysis[1])
            break
        else:
            print(f'skipped {st} -in s_w_a')
    #study_analaysises.append(study_analaysis)

study_analaysis = pd.concat(study_analaysises)

study_analaysis
"""


for study in studies_Ids:
    study_analaysis = GetAnalysisFromStudie(study)
    if(study_analaysis == None):
        continue
    study_analaysises.append(study_analaysis)


"""

[{'id': 'MGYS00005519', 'type': 'studies'}, {'id': 'MGYS00005520', 'type': 'studies'}]
statt MGYS00005519
get analysis from MGYS00005519
[{'id': 'MGYS00000498', 'type': 'studies'}]
statt MGYS00000498
get analysis from MGYS00000498
[{'id': 'MGYS00000498', 'type': 'studies'}]
statt MGYS00000498
skipped MGYS00000498 -duplicate- E
[{'id': 'MGYS00000498', 'type': 'studies'}]
statt MGYS00000498
skipped MGYS00000498 -duplicate- E
[{'id': 'MGYS00000498', 'type': 'studies'}]
statt MGYS00000498
skipped MGYS00000498 -duplicate- E
[{'id': 'MGYS00000498', 'type': 'studies'}]
statt MGYS00000498
skipped MGYS00000498 -duplicate- E
[{'id': 'MGYS00000498', 'type': 'studies'}]
statt MGYS00000498
skipped MGYS00000498 -duplicate- E
[{'id': 'MGYS00000498', 'type': 'studies'}]
statt MGYS00000498
skipped MGYS00000498 -duplicate- E
[{'id': 'MGYS00000498', 'type': 'studies'}]
statt MGYS00000498
skipped MGYS00000498 -duplicate- E
[{'id': 'MGYS00000498', 'type': 'studies'}]
statt MGYS00000498
skipped MGYS00000498

'\n\n\nfor study in studies_Ids:\n    study_analaysis = GetAnalysisFromStudie(study)\n    if(study_analaysis == None):\n        continue\n    study_analaysises.append(study_analaysis)\n\n\n'

In [13]:
study_analaysis

,type,id,attributes.accession,attributes.analysis-status,attributes.experiment-type,attributes.pipeline-version,attributes.analysis-summary,attributes.is-private,attributes.complete-time,attributes.instrument-platform,attributes.instrument-model,relationships.study.data.id,relationships.study.data.type,relationships.sample.data.id,relationships.sample.data.type,relationships.assembly.data.id,relationships.assembly.data.type,url,relationships.run.data.id,relationships.run.data.type
0,analysis-jobs,MGYA00678807,MGYA00678807,completed,assembly,5.0,"[{'key': 'Submitted nucleotide sequences', 'va...",False,2023-09-15T16:32:07,ILLUMINA,Illumina MiSeq,MGYS00005519,studies,ERS693376,samples,ERZ18322102,assemblies,https://www.ebi.ac.uk/metagenomics/api/v1/anal...,NaN,NaN
0,analysis-jobs,MGYA00678808,MGYA00678808,completed,assembly,5.0,"[{'key': 'Submitted nucleotide sequences', 'va...",False,2023-09-15T16:34:44,ILLUMINA,Illumina MiSeq,MGYS00005519,studies,ERS693376,samples,ERZ1761223,assemblies,https://www.ebi.ac.uk/metagenomics/api/v1/anal...,NaN,NaN
0,analysis-jobs,MGYA00528214,MGYA00528214,completed,assembly,4.1,"[{'key': 'Submitted nucleotide sequences', 'va...",False,2020-05-31T09:53:58,ILLUMINA,Illumina MiSeq,MGYS00005519,studies,ERS693376,samples,ERZ1761223,assemblies,https://www.ebi.ac.uk/metagenomics/api/v1/anal...,NaN,NaN
0,analysis-jobs,MGYA00651527,MGYA00651527,completed,metagenomic,5.0,"[{'key': 'Submitted nucleotide sequences', 'va...",False,2023-08-06T03:36:08,ILLUMINA,Illumina HiSeq 2500,MGYS00000498,studies,ERS745059,samples,NaN,NaN,https://www.ebi.ac.uk/metagenomics/api/v1/anal...,ERR911383,runs
0,analysis-jobs,MGYA00651534,MGYA00651534,completed,metagenomic,5.0,"[{'key': 'Submitted nucleotide sequences', 'va...",False,2023-08-06T03:47:29,ILLUMINA,Illumina HiSeq 2500,MGYS00000498,studies,ERS745058,samples,NaN,NaN,https://www.ebi.ac.uk/metagenomics/api/v1/anal...,ERR911382,runs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,analysis-jobs,MGYA00007787,MGYA00007787,completed,metagenomic,2.0,"[{'key': 'Submitted nucleotide sequences', 'va...",False,2015-07-22T00:00:00,ILLUMINA,Illumina HiSeq 2500,MGYS00000498,studies,ERS744990,samples,NaN,NaN,https://www.ebi.ac.uk/metagenomics/api/v1/anal...,ERR911314,runs
0,analysis-jobs,MGYA00007788,MGYA00007788,completed,metagenomic,2.0,"[{'key': 'Submitted nucleotide sequences', 'va...",False,2015-07-22T00:00:00,ILLUMINA,Illumina HiSeq 2500,MGYS00000498,studies,ERS744993,samples,NaN,NaN,https://www.ebi.ac.uk/metagenomics/api/v1/anal...,ERR911317,runs
0,analysis-jobs,MGYA00007789,MGYA00007789,completed,metagenomic,2.0,"[{'key': 'Submitted nucleotide sequences', 'va...",False,2015-07-22T00:00:00,ILLUMINA,Illumina HiSeq 2500,MGYS00000498,studies,ERS744992,samples,NaN,NaN,https://www.ebi.ac.uk/metagenomics/api/v1/anal...,ERR911316,runs
0,analysis-jobs,MGYA00007790,MGYA00007790,completed,metagenomic,2.0,"[{'key': 'Submitted nucleotide sequences', 'va...",False,2015-07-22T00:00:00,ILLUMINA,Illumina HiSeq 2500,MGYS00000498,studies,ERS744970,samples,NaN,NaN,https://www.ebi.ac.uk/metagenomics/api/v1/anal...,ERR911294,runs


In [14]:
analysis_accesions = []

for sample_acc in sample_accessions:
    acc = study_analaysis[study_analaysis['relationships.sample.data.id'] == sample_acc]['attributes.accession'].to_list()
    analysis_accesions.append([len(acc),acc])

analysis_accesions

[[3, ['MGYA00678807', 'MGYA00678808', 'MGYA00528214']],
 [3, ['MGYA00652169', 'MGYA00143092', 'MGYA00007718']],
 [3, ['MGYA00652168', 'MGYA00143011', 'MGYA00007717']],
 [3, ['MGYA00652167', 'MGYA00142989', 'MGYA00007716']],
 [3, ['MGYA00652166', 'MGYA00142985', 'MGYA00007723']],
 [3, ['MGYA00652165', 'MGYA00143083', 'MGYA00007722']],
 [3, ['MGYA00652164', 'MGYA00143076', 'MGYA00007721']],
 [3, ['MGYA00652163', 'MGYA00142998', 'MGYA00007720']],
 [3, ['MGYA00652162', 'MGYA00142981', 'MGYA00007773']],
 [3, ['MGYA00652161', 'MGYA00143017', 'MGYA00007726']]]

In [15]:

for analysis_sample in analysis_accesions:
    url = ""
    for analysis_id in analysis_sample[1]:
        print(f"start grab for {analysis_id}")
        #########################################
        # Get all downloads for one analysis
        #########################################

        with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
            
            dfs = []
            for r in mgnify.iterate(f'analyses/{analysis_id}/downloads'):
                df = pd.json_normalize(r.json)
                df['url'] = str(r.links.self)
                dfs.append(df)
            
        main_df = pd.concat(dfs)

        #print(main_df)

        ##########################
        # get specific data url
        #########################

        data_type = "TSV"
        data_label = "OTUs and taxonomic assignments for SSU rRNA"

        c1 = main_df["attributes.file-format.name"] == data_type
        c2 = main_df["attributes.description.description"] == data_label
        if(main_df.loc[(c1 & c2), "url"].size == 0):
            continue
        url = main_df.loc[(c1 & c2), "url"].iloc[0]
        break

    if(url == ""):
        print(f'no analysis found for {analysis_sample}')
        continue

    ##########################
    # download the data
    #########################

    data_output_folder = 'outputs/collection'
    os.makedirs(data_output_folder, exist_ok=True)

    response = requests.get(url)

    if not response:
        print(f"Could not download file, got response: {response.status_code}")
        break
        
    print(url)

    data_output_path = os.path.join(data_output_folder, f"{analysis_id}.tsv")
    with open(data_output_path, "w") as f:
        f.write(response.text)
        
    # add to galaxy (manually)
    #put(data_output_path)
     

start grab for MGYA00678807


https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00678807/file/ERZ18322102_FASTA_SSU_OTU.tsv
start grab for MGYA00652169
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00652169/file/ERR911201_MERGED_FASTQ_SSU_OTU.tsv
start grab for MGYA00652168
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00652168/file/ERR911202_MERGED_FASTQ_SSU_OTU.tsv
start grab for MGYA00652167
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00652167/file/ERR911203_MERGED_FASTQ_SSU_OTU.tsv
start grab for MGYA00652166
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00652166/file/ERR911204_MERGED_FASTQ_SSU_OTU.tsv
start grab for MGYA00652165
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00652165/file/ERR911205_MERGED_FASTQ_SSU_OTU.tsv
start grab for MGYA00652164
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00652164/file/ERR911206_MERGED_FASTQ_SSU_OTU.tsv
start grab for MGYA00652163
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00652163/file/ERR911207_ME

In [17]:
from jsonapi_client import Session
import pandas as pd
import numpy as np
import requests
import os

# MGYA00678807
analysis_sample = ['MGYA00452486']


for analysis_id in analysis_sample:
    print(f"start grab for {analysis_id}")
        #########################################
        # Get all downloads for one analysis
        #########################################

    with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
            
        dfs = []
        for r in mgnify.iterate(f'analyses/{analysis_id}/downloads'):
            df = pd.json_normalize(r.json)
            df['url'] = str(r.links.self)
            dfs.append(df)
            
    main_df = pd.concat(dfs)

    #print(main_df)

        ##########################
        # get specific data url
        #########################

    data_type = "TSV"
    data_label = "OTUs and taxonomic assignments for SSU rRNA"
    c1 = main_df["attributes.file-format.name"] == data_type
    c2 = main_df["attributes.description.description"] == data_label
    main_df = main_df.loc[(c1)]
main_df

start grab for MGYA00452486


,type,id,attributes.alias,attributes.file-format.name,attributes.file-format.extension,attributes.file-format.compression,attributes.description.label,attributes.description.description,attributes.group-type,attributes.file-checksum.checksum,attributes.file-checksum.checksum-algorithm,relationships.pipeline.data.id,relationships.pipeline.data.type,url
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_InterPro_1.tsv.gz,SRR5788294_MERGED_FASTQ_InterPro_1.tsv.gz,TSV,tsv,True,InterPro matches,InterPro matches (TSV),Functional analysis,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_InterPro_2.tsv.gz,SRR5788294_MERGED_FASTQ_InterPro_2.tsv.gz,TSV,tsv,True,InterPro matches,InterPro matches (TSV),Functional analysis,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_InterPro_3.tsv.gz,SRR5788294_MERGED_FASTQ_InterPro_3.tsv.gz,TSV,tsv,True,InterPro matches,InterPro matches (TSV),Functional analysis,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_SSU_MAPSeq.mseq.gz,SRR5788294_MERGED_FASTQ_SSU_MAPSeq.mseq.gz,TSV,tsv,True,MAPseq SSU assignments,MAPSeq output file for SSU,Taxonomic analysis SSU rRNA,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_SSU_OTU.tsv,SRR5788294_MERGED_FASTQ_SSU_OTU.tsv,TSV,tsv,False,Reads encoding SSU rRNA,All reads encoding SSU rRNA,Taxonomic analysis SSU rRNA,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_LSU_MAPSeq.mseq.gz,SRR5788294_MERGED_FASTQ_LSU_MAPSeq.mseq.gz,TSV,tsv,True,MAPseq LSU assignments,MAPSeq output file for LSU,Taxonomic analysis LSU rRNA,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_LSU_OTU.tsv,SRR5788294_MERGED_FASTQ_LSU_OTU.tsv,TSV,tsv,False,Reads encoding LSU rRNA,All reads encoding LSU rRNA,Taxonomic analysis LSU rRNA,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
